# ARGO Date from 30.12.2017

We use the position and temperature from the argo float array:

 * http://www.argo.ucsd.edu/
 * Argo (2000). Argo float data and metadata from Global Data Assembly Centre (Argo GDAC). SEANOE. http://doi.org/10.17882/42182
  
The data was obtained from
 * ftp://ftp.ifremer.fr/ifremer/argo/geo



In [ ]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:


def process_argo_data(ocean, date):
    
    fN = '../../rawdata/{}_{}_prof.nc'.format(ocean, date)
    
    ds = xr.open_dataset(fN)
    
    # only select lon, lat, temp
    ds = ds[['LONGITUDE', 'LATITUDE', 'TEMP']]
    
    # only the topmost layer
    ds = ds.isel(N_LEVELS=0)
    
    ds = ds.rename(dict(LATITUDE='lat', LONGITUDE='lon'))

    fN_out = 'ARGO_{}_{}.nc'.format(ocean, date)

    ds.to_netcdf(fN_out, format='NETCDF4_CLASSIC')
    
    return ds


ATL = process_argo_data('ATL', '20171230')
IND = process_argo_data('IND', '20171230')
PAC = process_argo_data('PAC', '20171230')
